In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.1


In [2]:
!nvidia-smi

Tue Apr 23 01:31:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P0              29W / 115W |     11MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
print(torch.__config__.show())

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO 

In [6]:
import doxapy
from tabulate import tabulate

def evaluate(gt_imgs, pr_imgs, print_results=True):
    perf_data = []

    for gt_img, pr_img in zip(gt_imgs, pr_imgs):
        perf_data.append(doxapy.calculate_performance(gt_img, pr_img))

    mean_values = {}

    for d in perf_data:
        for key, value in d.items():
            if key not in mean_values:
                mean_values[key] = []
            mean_values[key].append(value)
        
    nice_names = {
        'accuracy': 'Accuracy (%)',
        'fm': 'F-measure',
        'mcc': 'Matthews Correlation Coefficient',
        'psnr': 'Peak Signal-to-Noise Ratio (PSNR)',
        'nrm': 'Normalized Root Mean Square Error (NRM)',
        'drdm': 'Distance-based Performance Measure (DRDM)'
    }

    mean_values_single = {nice_names[key]: round(value[0], 2) for key, value in mean_values.items()}
    
    if(print_results == True):
        data = [
            ["Metric", "Processed Images"],
            ["Accuracy (%)", "{:.2f}".format(mean_values_single['accuracy'])],
            ["F-measure", "{:.2f}".format(mean_values_single['fm'])],
            ["Matthews Correlation Coefficient", "{:.2f}".format(mean_values_single['mcc'])],
            ["Peak Signal-to-Noise Ratio (PSNR)", "{:.2f}".format(mean_values_single['psnr'])],
            ["Normalized Root Mean Square Error (NRM)", "{:.2f}".format(mean_values_single['nrm'])],
            ["Distance-based Performance Measure (DRDM)", "{:.2f}".format(mean_values_single['drdm'])],
        ]

        print(tabulate(data, headers="firstrow", tablefmt="grid"))

    return mean_values_single

def show_metrics_table(metrics_dicts):
    metrics_names = list(metrics_dicts.values())[0].keys()

    headers = ["Metric"] + list(metrics_dicts.keys())

    table_data = []

    for metric_name in metrics_names:
        row_data = [metric_name]
        for metrics_dict in metrics_dicts.values():
            row_data.append("{:.2f}".format(metrics_dict[metric_name]))
        table_data.append(row_data)

    print(tabulate(table_data, headers=headers, tablefmt="grid"))

In [8]:
from tabulate import tabulate

def evaluate(gt_imgs, pr_imgs, print_results=True):
    perf_data = []

    # Calculate performance for each ground truth and predicted image pair
    for gt_img, pr_img in zip(gt_imgs, pr_imgs):
        perf_data.append(doxapy.calculate_performance(gt_img, pr_img))

    mean_values = {}

    # Aggregate values for each metric across all image comparisons
    for d in perf_data:
        for key, value in d.items():
            if key not in mean_values:
                mean_values[key] = []
            mean_values[key].append(value)
        
    # Prepare nicer names for the metrics to display
    nice_names = {
        'accuracy': 'Accuracy (%)',
        'fm': 'F-measure',
        'mcc': 'Matthews Correlation Coefficient',
        'psnr': 'Peak Signal-to-Noise Ratio (PSNR)',
        'nrm': 'Normalized Root Mean Square Error (NRM)',
        'drdm': 'Distance-based Performance Measure (DRDM)'
    }

    # Prepare a dictionary with the average values for each metric, if present
    mean_values_single = {}
    for key, values in mean_values.items():
        if key in nice_names:
            mean_values_single[nice_names[key]] = round(sum(values) / len(values), 2)

    # Print results if specified
    if print_results:
        data = [["Metric", "Processed Images"]]
        for metric_name, metric_value in mean_values_single.items():
            data.append([metric_name, "{:.2f}".format(metric_value)])

        print(tabulate(data, headers="firstrow", tablefmt="grid"))

    return mean_values_single

In [9]:
import cv2
# from assignment_1.binarization.evaluation import evaluate
gt_path = 'data/out/true_119.png'
gt_path1 = 'data/out/20_gt.bmp'
pr_path1 = 'data/out/20.png'
pr_path = 'data/out/pred_119.png'
gt_img = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
gt_img1 = cv2.imread(gt_path1, cv2.IMREAD_GRAYSCALE)
pr_img = cv2.imread(pr_path, cv2.IMREAD_GRAYSCALE)
pr_img1 = cv2.imread(pr_path1, cv2.IMREAD_GRAYSCALE)
gt_imgs = [gt_img, gt_img1]
pr_imgs = [pr_img, pr_img1]

evaluate(gt_imgs, pr_imgs, print_results=True)

+-------------------------------------------+--------------------+
| Metric                                    |   Processed Images |
+===========================================+====================+
| Accuracy (%)                              |              95.51 |
+-------------------------------------------+--------------------+
| F-measure                                 |              82.37 |
+-------------------------------------------+--------------------+
| Matthews Correlation Coefficient          |               0.81 |
+-------------------------------------------+--------------------+
| Peak Signal-to-Noise Ratio (PSNR)         |              13.55 |
+-------------------------------------------+--------------------+
| Normalized Root Mean Square Error (NRM)   |               0.1  |
+-------------------------------------------+--------------------+
| Distance-based Performance Measure (DRDM) |               4.65 |
+-------------------------------------------+-----------------

{'Accuracy (%)': 95.51,
 'F-measure': 82.37,
 'Matthews Correlation Coefficient': 0.81,
 'Peak Signal-to-Noise Ratio (PSNR)': 13.55,
 'Normalized Root Mean Square Error (NRM)': 0.1,
 'Distance-based Performance Measure (DRDM)': 4.65}